In [3]:
import pandas as pd
import datetime
import src.support as sp
import h2o
from h2o.automl import H2OAutoML
# inicializar H2O
h2o.init()

h2o:1: DeprecationWarning: Your Python version is 3.11.2. The support for this version will be removed in H2O 3.42.0.1.


Checking whether there is an H2O instance running at http://localhost:54321. connected.


H2O_cluster_uptime:,1 hour 51 mins
H2O_cluster_timezone:,Europe/Madrid
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.40.0.3
H2O_cluster_version_age:,17 days
H2O_cluster_name:,H2O_from_python_juano_qk5ryf
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.126 Gb
H2O_cluster_total_cores:,16
H2O_cluster_allowed_cores:,16
H2O_cluster_status:,"locked, healthy"


In [5]:
data4 = sp.call_csv(ruta='../data/merged_df_2022.csv').drop('revenue_resold', axis=1)
data4.columns

Index(['date', 'transaction_type_name', 'brand_id_name', 'format_name',
       'device_name', 'media_type', 'industria', 'impressions_resold', 'cpm',
       'media_prec', 'sessions', 'avgSessionDuration', 'users'],
      dtype='object')

In [6]:
data4.head(1)

,date,transaction_type_name,brand_id_name,format_name,device_name,media_type,industria,impressions_resold,cpm,media_prec,sessions,avgSessionDuration,users
0,2022-02-01,Preferred deal,FYKGKT8,Doble Robapaginas,Mobile,Web,BUSINESS & INDUSTRIAL : Retail - National,7090940,4.27,9.980189,28352276.0,104.732242,6734689.0


In [28]:
data2 = sp.call_csv(ruta='../data/merged_df_2022_date.csv')
data2['date'] =  pd.to_datetime(data2['date'], format='%Y-%m-%d').dt.date

selected_date = (datetime.date(2022, 2, 1))

# Filtra la fecha seleccionada en mi df por la columna date
filtered_data = data2[data2["date"] == selected_date]

if not filtered_data.empty:
    # Mostrar las métricas de los datos filtrados
    print(filtered_data["media_prec"].round(2).iloc[0])
  
else:
    print(f"No se encontraron datos para la fecha seleccionada: {selected_date}")

9.98


In [40]:
data = sp.call_csv(ruta='../data/merged_df_2022_formatos.csv')
data['date'] =  pd.to_datetime(data['date'], format='%Y-%m-%d').dt.date
a = data[data['date']==datetime.date(2022, 2, 1)]
a

,format_name,date,revenue_resold
0,300x600 (Half Page),2022-02-01,40853.85
23,320x50 (Mobile Banner),2022-02-01,4826.10
42,300x250 (Medium Rectangle),2022-02-01,2374.84
51,Video,2022-02-01,1476.16
71,970x250 (Billboard),2022-02-01,820.26
74,Interstitial,2022-02-01,800.40
80,320x100 (Mobile Double Size Banner),2022-02-01,625.66
103,728x90 (Leaderboard - Mobile Banner Landscape),2022-02-01,240.60
107,1800x1000 (Wallpaper/Skin),2022-02-01,182.55
113,Animated Skin VPAID,2022-02-01,80.25


In [66]:

data = sp.call_csv(ruta='../data/merged_df_2022_brand_name.csv')
data['date'] =  pd.to_datetime(data['date'], format='%Y-%m-%d').dt.date
a = data[data['date']==datetime.date(2022, 2, 1)]
a

,brand_id_name,date,revenue_resold
0,M4X8929,2022-02-01,37255.67
10,2EP267U,2022-02-01,6931.25
30,EIYJO7L,2022-02-01,3804.47
70,TI4IKGR,2022-02-01,1080.01
79,X9PVXP5,2022-02-01,891.56
80,M4IGGTN,2022-02-01,875.75
85,HCVPAS7,2022-02-01,617.75
90,QPT8JUL,2022-02-01,545.04
102,TC04QCB,2022-02-01,270.63
130,7QCVJP2,2022-02-01,19.84


In [63]:
def plot_histplot(df):

    filtered_data = df[df["date"] == datetime.date(2022, 2, 1)]
    # Create the histogram using Plotly Express
    fig = px.histogram(filtered_data, y='revenue_resold', x='format_name')

    # Remove the y-axis label
    fig.update_layout(yaxis=dict(title=None))

    # Remove the y-axis ticks
    fig.update_yaxes(showticklabels=False)

    # Return the plot
    return fig


In [64]:
plot_histplot(data)

In [7]:
def predict_with_best_model(aml_path, test_path):
    # Load the best H2O AutoML model
    best_model = h2o.load_model(aml_path)

    # Import the new data as an H2OFrame
    new_data = h2o.import_file(test_path)

    # Predict using the loaded model
    predictions = best_model.predict(new_data)

    # Convert predictions to a pandas DataFrame
    predictions_df = predictions.as_data_frame()

    # Return the predictions DataFrame
    return predictions_df

In [12]:
# Cargar el mejor modelo en memoria
# ==============================================================================

loaded_model = h2o.load_model("../data/best_model_3/DeepLearning_grid_2_AutoML_2_20230421_201630_model_1")

# Predecimos 
# ==============================================================================

# Cargar los nuevos datos de entrada como un objeto H2OFrame
new_data = h2o.import_file(path = "../data/df_test.csv")

# Hacer predicciones en los nuevos datos utilizando el modelo cargado
predictions = loaded_model.predict(new_data)

predictions_df = predictions.as_data_frame()


resultado = predictions_df['predict'][0].round(2)

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
deeplearning prediction progress: |██████████████████████████████████████████████| (done) 100%


In [13]:
resultado

16727.99

In [14]:
nuevo_df= pd.DataFrame(columns=data4.columns)
nuevo_df = {'date':selected_date2, 'transaction_type_name':option, 'brand_id_name':option2, 'format_name':option3,
       'device_name':option4, 'media_type':option5, 'industria':option6, 'impressions_resold':number, 'cpm':number2,
       'media_prec':number3, 'sessions':number4, 'avgSessionDuration':number5, 'users':number6}

NameError: name 'selected_date2' is not defined